In [ ]:
import sys, os  
import pandas as pd  
import numpy as np  

from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D  
from keras.losses import categorical_crossentropy  
from keras.optimizers import Adam  
from keras.regularizers import l2  
from keras.utils import np_utils 

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Data Kaggle/Emotion Detection/fer2013.csv')  
X_train,train_y,X_test,test_y=[],[],[],[]  
for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    try:  
        if 'Training' in row['Usage']:  
           X_train.append(np.array(val,'float32'))  
           train_y.append(row['emotion'])  
        elif 'PublicTest' in row['Usage']:  
           X_test.append(np.array(val,'float32'))  
           test_y.append(row['emotion'])  
    except:  
        print(f"error occured at index :{index} and row:{row}") 

In [ ]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 20  
width, height = 48, 48  

X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)   
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)  

X_train -= np.mean(X_train, axis=0) #normalize dữ liệu giữa 0 và 1 
X_train /= np.std(X_train, axis=0)  

X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)  

In [ ]:
model = Sequential()  

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(Conv2D(64, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(Conv2D(128, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  

model.add(Flatten())  

#fully connected neural networks  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  

model.add(Dense(num_labels, activation='softmax'))  

# model.summary()  

#Compliling the model  
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy']) 

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 9, 9, 64)         

In [ ]:
#Training the model  
model.fit(X_train, train_y,  
          batch_size=batch_size,  
          epochs=30,  
          verbose=1,  
          validation_data=(X_test, test_y),  
          shuffle=True)  

Epoch 1/30
449/449 [==============================] - 9s 19ms/step - loss: 1.7892 - accuracy: 0.2611 - val_loss: 1.5703 - val_accuracy: 0.3722
Epoch 2/30
449/449 [==============================] - 8s 18ms/step - loss: 1.5496 - accuracy: 0.3862 - val_loss: 1.4162 - val_accuracy: 0.4402
Epoch 3/30
449/449 [==============================] - 8s 19ms/step - loss: 1.4101 - accuracy: 0.4520 - val_loss: 1.3135 - val_accuracy: 0.4870
Epoch 4/30
449/449 [==============================] - 9s 19ms/step - loss: 1.3431 - accuracy: 0.4819 - val_loss: 1.2779 - val_accuracy: 0.5116
Epoch 5/30
449/449 [==============================] - 8s 19ms/step - loss: 1.2937 - accuracy: 0.5056 - val_loss: 1.2604 - val_accuracy: 0.5166
Epoch 6/30
449/449 [==============================] - 9s 19ms/step - loss: 1.2487 - accuracy: 0.5232 - val_loss: 1.2342 - val_accuracy: 0.5241
Epoch 7/30
449/449 [==============================] - 9s 19ms/step - loss: 1.2202 - accuracy: 0.5295 - val_loss: 1.2072 - val_accuracy: 0.5344

In [ ]:
#Saving the  model 
fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("fer.h5")  